In [1]:
import pandas as pd
import numpy as np
import matplotlib
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

pd.set_option("display.precision", 8)

In [2]:

output_csv_dir = '../output/csv'
output_pickle_dir = '../input/pickle'

if(False):
    for path, subdirs, files in os.walk(output_csv_dir):
        for name in files:        
            df_name = name.split('.')[0] #get df name from filename
            df = pd.read_csv(os.path.join(path, name))        
            df.to_pickle(os.path.join(output_pickle_dir, df_name))

In [3]:
Alerts = pd.read_pickle(os.path.join(output_pickle_dir, 'Alerts'))
dP_siteSummarised = pd.read_pickle(os.path.join(output_pickle_dir, 'dP_siteSummarised'))
iP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'iP_Summarised'))
mP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'mP_Summarised'))
#sP_shiftData = pd.read_pickle(os.path.join(output_pickle_dir, 'MTP_WF003_05_Visualise Baseline Plan_ShiftData'))
sP_portSummarisedCofill = pd.read_pickle(os.path.join(output_pickle_dir, 'sP_portSummarisedCofill'))
sP_portSummarisedIH = pd.read_pickle(os.path.join(output_pickle_dir, 'sP_portSummarisedIH'))
sP_portSummarisedIHLineLoading = pd.read_pickle(os.path.join(output_pickle_dir, 'sP_portSummarisedIHLineLoading'))


In [90]:
dP_siteSummarised[(dP_siteSummarised.Site_SAP == '5502') 
                  & (dP_siteSummarised.Year_Period == '2020_12') 
                  & (dP_siteSummarised.Material == '203380')
                  & (dP_siteSummarised.Uom == 'ZUC')
                  & (dP_siteSummarised.KeyField == 'Total')
                 ]['Value'].values[0]

1003.0

In [105]:
dP_siteSummarised.Material[0:10].str.cat(sep="','")

"203380','203380','203380','203380','203381','203381','203381','203381','203475','203475"

In [38]:
iP_Summarised.head()

,Table,Level2Region,Site_SAP,SNPPL01,KeyField,Value,Uom,SiteCountry,Summary_Portfolio,Material,Line_Portfolio,Year_Period,DSI_Type
0,Inventory_MaterialLevel,,AS&D,NRB GLASS (IH),GBP_ClosingCostOfCapital_Sum,0.00000000,GBP,GB,Glass,201052,Glass,2021_09,DSI_Other
1,Inventory_MaterialLevel,,AS&D,NRB GLASS (IH),GBP_ClosingStockValue_Avg,0.00000000,GBP,GB,Glass,201052,Glass,2021_09,DSI_Other
2,Inventory_MaterialLevel,,AS&D,NRB GLASS (IH),GBP_CostOfCapital_Sum,0.00000000,GBP,GB,Glass,201052,Glass,2021_09,DSI_Other
3,Inventory_MaterialLevel,,AS&D,NRB GLASS (IH),GBP_StockValue_Avg,0.00000000,GBP,GB,Glass,201052,Glass,2021_09,DSI_Other
4,Inventory_MaterialLevel,,AS&D,NRB GLASS (IH),PAL_ClosingStock,0.86749022,PAL,GB,Glass,201052,Glass,2021_09,DSI_Other


In [41]:
inventory_excess = pd.pivot_table(df, values=value_col, index=pivot_rows, columns=pivot_cols, aggfunc=agg_func)
inventory_excess

Value
DSI_Type                   DSI_Other
Line_Portfolio Year_Period          
Aseptic        2020_12       99157.0
               2021_01       84664.0
               2021_02      101208.0
               2021_03      115958.0
               2021_04      137924.0
...                              ...
Water          2021_12      575168.0
               2022_01      655865.0
               2022_02      813239.0
               2022_03      793103.0
               2022_04      796509.0

[226 rows x 1 columns]

In [49]:
def get_next_period(this_period) :
    this_month = int(this_period.split('_')[1])
    this_year = int(this_period.split('_')[0])        
    
    next_year = this_year
    if this_month==12:
        next_year = this_year+1
        next_month = 1
    else:
        next_month = this_month+1
   
    return f"{next_year}_{next_month:02d}"

In [51]:
xx = [ get_next_period(x) for x in  inventory_excess.Year_Period ]
yy = inventory_excess.Year_Period 

print(xx[1:10])

print(yy[1:10])

['2021_02', '2021_03', '2021_04', '2021_05', '2021_06', '2021_07', '2021_08', '2021_09', '2021_10']
1    2021_01
2    2021_02
3    2021_03
4    2021_04
5    2021_05
6    2021_06
7    2021_07
8    2021_08
9    2021_09
Name: Year_Period, dtype: object


In [63]:
pivot_cols = ['KeyField']
pivot_rows = ['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom': 'ZUC'}

df = iP_Summarised[iP_Summarised.KeyField.isin(['ZUC_StockVsTarget'])]

inventory_excess = pd.pivot_table(df, values=value_col, index=pivot_rows, columns=pivot_cols, aggfunc=agg_func)

inventory_excess = pd.DataFrame(inventory_excess.to_records())

inventory_excess.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in inventory_excess.columns]

inventory_excess.Year_Period = [ get_next_period(x) for x in  inventory_excess.Year_Period ]

inventory_excess.columns = [pivot_rows + ['Excess_Inventory']]

num = inventory_excess._get_numeric_data()
num[num < 0] = 0


inventory_excess.loc[inventory_excess.Excess_Inventory < 0]

TypeError: only integer scalar arrays can be converted to a scalar index

In [4]:
INPUT_PICKLE_DIR = '../input/pickle'
SUPPLY_PLAN_INHOUSE_LINELOADING_PICKLE = 'sP_portSummarisedIHLineLoading'

level='Line_Portfolio'
uom='Hours'

In [5]:
def summarise_df_for_visualisation(df):
    unwanted_cols = ['Material', 'Value']
    cols = [ele for ele in df.columns.tolist() if ele not in unwanted_cols] 
    
    df['Value'] = df['Value'].fillna(0)
    df[cols] = df[cols].fillna('') 
        
    df = df[cols+['Value']].groupby(cols).sum().round().reset_index()  
    
    return df


In [6]:
def get_production_line_plan(level='Line_Portfolio', uom='Hours'):
    pivot_cols = ['KeyField']
    pivot_rows = [level, 'Site_SAP', 'Line', 'Year_Period']
    value_col = ['Value']
    agg_func = np.sum
    filter_by = {'Uom': uom}

    df = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, SUPPLY_PLAN_INHOUSE_LINELOADING_PICKLE))
    
    df = summarise_df_for_visualisation(df)

    wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                        columns=pivot_cols, aggfunc=agg_func)

    production_line_plan = pd.DataFrame(wide_df.to_records())

    production_line_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                        for hdr in production_line_plan.columns]

    production_line_plan['Consumed'] = production_line_plan['Consumed'].astype(float)
    production_line_plan['Capacity'] = production_line_plan['Capacity'].astype(float)

    #Calcuated fields
    production_line_plan['Capacity_Utilisation'] = production_line_plan['Consumed']*100 / production_line_plan['Capacity']

    production_line_plan['Available_Vol'] = production_line_plan['EstimatedUnconsumedCapacity']
    production_line_plan['Available_Vol_with_Safety_Hours'] = production_line_plan['EstimatedUnconsumedCapacity'] + production_line_plan['EstimatedHoursSpareCapacity']


    production_line_plan = production_line_plan.fillna(0)

    return production_line_plan

In [ ]:
pd.pivot_table(df, values=value_col, index=pivot_rows,
                        columns=pivot_cols, aggfunc=agg_func)

In [4]:
sum(dP_siteSummarised.loc[(dP_siteSummarised.Material == '245692') & 
                          (dP_siteSummarised.KeyField == 'Total') & 
                          (dP_siteSummarised.Uom == 'ZUC') & 
                          (dP_siteSummarised.Year_Period == '2021_01')].Value)

3902335.0

In [7]:
# Demand Plan

pivot_cols = ['KeyField']
pivot_rows = ['Summary_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}

df = dP_siteSummarised.loc[(dP_siteSummarised[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

plot_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

demand_plan = pd.DataFrame(plot_df.to_records())

demand_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                     for hdr in demand_plan.columns]

#Calcuated fields
demand_plan['Stock_Availability'] = (demand_plan['Available']*100 / demand_plan['Total']).astype(int)

demand_plan = demand_plan.fillna(0)


In [134]:
# Movement Plan

pivot_cols = ['Move_Type']
pivot_rows = ['Summary_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}
#filter_by = {'Uom':'ZUC', 'FromCountry':'', '':''}

mP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'mP_Summarised'))


cofill_sites = ['AILI_IE', 'APPL_GB', 'CACO_FR', 'COTT_GB', 'DAYL_GB', 'DIS_NL', 'EDRI_GB', 'KART_GE', 'MBEV_IN', 'MILK_GB', 'NORB_GB', 'PRIN_GB', 'WILD_GE', 'WINT_NL']
cofill_sites = [x.split('_')[0] for x in cofill_sites]

import_sites = ['15N', '19B', '44F']


# Conditional categories
conditions = [
    (mP_Summarised['FromCountry'] == 'GB') & (mP_Summarised['ToCountry'] != 'GB'),
    (mP_Summarised['Site_SAP_From'].isin(cofill_sites)),
    (mP_Summarised['Site_SAP_From'].isin(import_sites)),
    (mP_Summarised['FromCountry'] == 'GB') & (mP_Summarised['ToCountry'] == 'GB') & (mP_Summarised['ToRepack_Flag'] == 'True'),
    (mP_Summarised['FromCountry'] == 'GB') & (mP_Summarised['ToCountry'] == 'GB') & (mP_Summarised['ToRepack_Flag'] == 'False')
    ]
choices = ['Export', 'Cofill', 'Import', 'Domestic Repack', 'IBT']

mP_Summarised['Move_Type'] = np.select(conditions, choices, default='Move_Other')


df = mP_Summarised.loc[(mP_Summarised[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

plot_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

movement_plan = pd.DataFrame(plot_df.to_records())

movement_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in movement_plan.columns]

movement_plan = movement_plan.fillna(0)

In [135]:
movement_plan

,Summary_Portfolio,Year_Period,Cofill,Export,IBT,Import,Move_Other
0,Aseptic,2020_12,0.0,108731.79,17185.0,57717.80,0.0
1,Aseptic,2021_01,7892.0,88671.21,38355.0,64429.22,0.0
2,Aseptic,2021_02,27725.0,95561.60,64133.0,72707.60,0.0
3,Aseptic,2021_03,40515.0,118730.58,93792.0,64684.18,0.0
4,Aseptic,2021_04,31361.0,112914.90,81539.0,78749.80,0.0
...,...,...,...,...,...,...,...
332,Water,2021_12,0.0,9699.00,885324.0,0.00,0.0
333,Water,2022_01,0.0,8077.00,802007.0,0.00,0.0
334,Water,2022_02,0.0,8734.00,927809.0,0.00,0.0
335,Water,2022_03,0.0,11109.00,1202731.0,0.00,0.0


In [9]:
movement_plan.loc[(movement_plan.Summary_Portfolio == 'Capri Sun') &
                  (movement_plan.Year_Period == '2021_01')                  
                 ] 

,Summary_Portfolio,Year_Period,Cofill_Movement,Export,IBT,Import,Move_Other
83,Capri Sun,2021_01,183298.0,0.0,481171.0,0.0,0.0


In [10]:
mP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'mP_Summarised'))

mP_Summarised.loc[(mP_Summarised.Material == '245692') &
                  (mP_Summarised.Year_Period == '2021_01') &
                  (mP_Summarised.Uom == 'ZUC') 
                 ] 

,FromCountry,ToCountry,SiteCountry,From_GeoRegion,To_GeoRegion,Site_SAP_From,Site_SAP_To,Year_Period,Summary_Portfolio,Material,KeyField,Value,Uom,ToRepack_Flag
15756,GB,GB,,Scotland,North,5503,5502,2021_01,Capri Sun,245692,ZUC,4295.0,ZUC,False
20241,GB,GB,,Scotland,North,5503,5558,2021_01,Capri Sun,245692,ZUC,4564.0,ZUC,False
25831,GB,GB,,Scotland,North,5503,5567,2021_01,Capri Sun,245692,ZUC,2109.0,ZUC,False
36731,GB,GB,,South,North,5501,5502,2021_01,Capri Sun,245692,ZUC,4144.0,ZUC,False
46581,GB,GB,,South,North,5501,5558,2021_01,Capri Sun,245692,ZUC,4403.0,ZUC,False
84626,GB,GB,,South,South,5501,5506,2021_01,Capri Sun,245692,ZUC,2109.0,ZUC,False
96581,GB,GB,,South,South,5501,5556,2021_01,Capri Sun,245692,ZUC,11604.0,ZUC,False
113941,GB,GB,,South,South,5501,5559,2021_01,Capri Sun,245692,ZUC,1581.0,ZUC,False


In [11]:
# Production Plan In House
uom = 'ZUC'
pivot_cols = ['Uom']
pivot_rows = ['Summary_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}

df = sP_portSummarisedIH.loc[(sP_portSummarisedIH[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

plot_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

prod_plan_inhouse = pd.DataFrame(plot_df.to_records())

prod_plan_inhouse.columns = [hdr.replace("('Value', '", "").replace(f"{uom}')", "Production") \
                     for hdr in prod_plan_inhouse.columns]

prod_plan_inhouse = prod_plan_inhouse.fillna(0)


In [12]:
# Production Plan  Cofill
uom = 'ZUC'
pivot_cols = ['Uom']
pivot_rows = ['Summary_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}

df = sP_portSummarisedCofill.loc[(sP_portSummarisedCofill[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

plot_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

prod_plan_cofill = pd.DataFrame(plot_df.to_records())

prod_plan_cofill.columns = [hdr.replace("('Value', '", "").replace(f"{uom}')", "Cofill") \
                     for hdr in prod_plan_cofill.columns]

prod_plan_cofill = prod_plan_cofill.fillna(0)

In [13]:
prod_plan_cofill.head()

,Summary_Portfolio,Year_Period,Cofill
0,Aseptic,2020_12,52368.0
1,Aseptic,2021_01,28833.0
2,Aseptic,2021_02,51654.0
3,Aseptic,2021_03,73170.0
4,Aseptic,2021_04,56946.0


In [14]:
# Inventory Plan 

pivot_cols = ['Inventory_Type']
pivot_rows = ['Summary_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}

iP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'iP_Summarised'))
# Conditional categories
conditions = [
    iP_Summarised.KeyField.str.endswith('ClosingStock'),
    iP_Summarised.KeyField.str.contains('StockVsTarget', case=False)
    ]
choices = ['Inventory_Closing','Inventory_National_Target']

iP_Summarised['Inventory_Type'] = np.select(conditions, choices, default='Inventory_Other')

df = iP_Summarised.loc[(iP_Summarised[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

plot_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

inventory_plan = pd.DataFrame(plot_df.to_records())

inventory_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in inventory_plan.columns]

inventory_plan = inventory_plan.fillna(0)

In [15]:
inventory_plan.loc[(inventory_plan.Summary_Portfolio == 'Capri Sun') &                                                     
                          (inventory_plan.Year_Period == '2021_01')]

,Summary_Portfolio,Year_Period,Inventory_Closing,Inventory_National_Target,Inventory_Other
69,Capri Sun,2021_01,583014.0,123.0,1154200.4


In [16]:
# Days of Stock Inventory Plan - Projected stock holding 

pivot_cols = ['DSI_Type']
pivot_rows = ['Line_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {}

iP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'iP_Summarised'))
# Conditional categories
conditions = [
    (iP_Summarised.KeyField == 'Days_DSI_National_LinePortfolio'),
    (iP_Summarised.KeyField == 'Days_DSI_Regional_LinePortfolio') & (iP_Summarised.Level2Region == 'North'),
    (iP_Summarised.KeyField == 'Days_DSI_Regional_LinePortfolio') & (iP_Summarised.Level2Region == 'South'),
    (iP_Summarised.KeyField == 'Days_DSI_Regional_LinePortfolio') & (iP_Summarised.Level2Region == 'Scotland'),
    (iP_Summarised.KeyField == 'Days_ClosingStockTarget_National_LinePortfolio')
    ]
choices = ['DSI_National','DSI_North', 'DSI_South', 'DSI_Scotland', 'DSI_Target_National']

iP_Summarised['DSI_Type'] = np.select(conditions, choices, default='DSI_Other')

df = iP_Summarised.loc[(iP_Summarised[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

plot_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

dsi_plan = pd.DataFrame(plot_df.to_records())

dsi_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in dsi_plan.columns]

dsi_plan = dsi_plan.fillna(0).round(2)

C:\Users\X80045674\.virtualenvs\mtp-P4lxf2bF\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [18]:
# Merge DFs into one

dfs = [demand_plan, movement_plan, prod_plan_inhouse, prod_plan_cofill, inventory_plan]
dfs = [df.set_index(['Summary_Portfolio', 'Year_Period']) for df in dfs]
combined_df = dfs[0].join(dfs[1:]).reset_index()
combined_df =combined_df.fillna(0)

#combined_df['Cofill'] = combined_df['Cofill'] + combined_df['Cofill_Movement'] 
#combined_df = combined_df.drop('Cofill_Movement', axis=1)

In [29]:
key_cols = ['Summary_Portfolio', 'Year_Period']
value_cols = list(set(combined_df.columns) - set(key_cols))

In [36]:
combined_df.loc[combined_df.Available == combined_df.Total]

,Summary_Portfolio,Year_Period,Available,Total,Stock_Availability,Cofill_Movement,Export,IBT,Import,Move_Other,Production,Cofill,Inventory_Closing,Inventory_National_Target,Inventory_Other
0,Aseptic,2020_12,100507.0,100507.0,100.0,0.0,108731.80,17185.0,57717.80,0.0,25397.0,52368.0,99157.0,20086.0,2.05960414e+05
1,Aseptic,2021_01,76587.0,76587.0,100.0,7892.0,88671.22,38355.0,64429.22,0.0,44835.0,28833.0,84664.0,1966.0,1.72698200e+05
2,Aseptic,2021_02,63142.0,63142.0,100.0,27725.0,95561.60,64133.0,72707.60,0.0,50884.0,51654.0,101208.0,120.0,1.96498500e+05
3,Aseptic,2021_03,101088.0,101088.0,100.0,40515.0,118730.58,93792.0,64684.18,0.0,96715.0,73170.0,115958.0,52.0,2.27321600e+05
4,Aseptic,2021_04,91946.0,91946.0,100.0,31361.0,112914.90,81539.0,78749.80,0.0,91131.0,56946.0,137924.0,15686.0,2.46863750e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Water,2021_12,821267.0,821267.0,100.0,0.0,9699.00,885324.0,0.00,0.0,802581.0,0.0,575168.0,0.0,1.29333063e+06
285,Water,2022_01,619945.0,619945.0,100.0,0.0,8077.00,802007.0,0.00,0.0,726491.0,0.0,655865.0,0.0,1.30677575e+06
286,Water,2022_02,709895.0,709895.0,100.0,0.0,8734.00,927809.0,0.00,0.0,896462.0,0.0,813239.0,60628.0,1.51514850e+06
287,Water,2022_03,1014121.0,1014121.0,100.0,0.0,11109.00,1202731.0,0.00,0.0,1031848.0,0.0,793103.0,-1.0,1.77054980e+06


In [31]:
long_df = combined_df.melt(id_vars=key_cols, 
              value_vars=value_cols,
              var_name='Category', value_name='Value')

long_df['Parent_Category'] = ['Demand' if x in ['Total','Domestic Repack', 'Export'] \
                                       else 'Supply' if x in ['Production','Import', 'Cofill'] \
                              else '' for x in long_df.Category]

In [32]:
long_df.Category.value_counts(dropna=False)

Available                    289
Move_Other                   289
Inventory_Closing            289
Cofill                       289
IBT                          289
Inventory_National_Target    289
Import                       289
Cofill_Movement              289
Stock_Availability           289
Total                        289
Inventory_Other              289
Export                       289
Production                   289
Name: Category, dtype: int64

In [33]:
long_df['Parent_Category'].value_counts(dropna=False)

          2023
Supply    1156
Demand     578
Name: Parent_Category, dtype: int64

In [22]:
labels={"Import": "Supply_Imports", 
        "Production": "Supply_Production",
       "Cofill": "Supply_Cofill",
       "Available": "Demand_Domestic_Sales",
       "Domestic Repack": "Demand_Domestic_Repack",
       "Export": "Demand_Export"}

In [23]:
plot_df = pd.merge(demand_supply_df, inventory_df, on=['Summary_Portfolio','Year_Period'], how='inner')


NameError: name 'demand_supply_df' is not defined

In [24]:
#'  Apply column names same as excel chart
#'  Separation between vertinal cols
#' Horizontal line


pd.options.display.float_format = '{:,.0f}'.format

demand_supply_df = long_df.loc[long_df.Parent_Category.isin(['Demand', 'Supply'])]
inventory_df =  combined_df[['Summary_Portfolio','Year_Period', 'Inventory_Closing','Inventory_National_Target','Inventory_Other']]

plot_df = pd.merge(demand_supply_df, inventory_df, on=['Summary_Portfolio','Year_Period'], how='inner')

plot_df = plot_df.groupby(['Year_Period', 'Parent_Category', 'Category']).sum().reset_index()

#plot_df = plot_df[plot_df.Summary_Portfolio == 'SPET']

plot_df = plot_df.round(0)

#fig = fig = make_subplots(specs=[[{"secondary_y": True}]]) #go.Figure()

fig = go.Figure()
fig = go.FigureWidget(fig)

fig.update_layout(
    template="simple_white",
    xaxis=dict(title_text="Year Period"),
    yaxis=dict(title_text="Units", tickformat=".2s"),
    barmode="stack"
    
)

colors = sns.color_palette("rocket").as_hex() #["#2A66DE","#fd5f42", "#c55f42", "#c58a42", "#c5c142", "#FFC32B"]

labels={"Import": "Supply_Imports", 
        "Production": "Supply_Production",
        "Cofill": "Supply_Cofill",
        "Total": "Demand_Domestic_Sales",
        "Domestic Repack": "Demand_Domestic_Repack",
        "Export": "Demand_Export"}

plot_df.Category = [labels[x] for x in plot_df.Category]
plot_df.sort_values(by=['Parent_Category', 'Category'])

categories = plot_df.Category.unique().tolist()
categories.sort()

for r, c in zip(categories, colors):  
    print(r)
    this_df = plot_df[plot_df.Category == r]    
    fig.add_trace(
        go.Bar(x=[this_df.Year_Period, this_df.Parent_Category], 
               y=this_df.Value, 
               marker_color=c,
               name = r               
              )
    )

plot_df = plot_df.sort_values(by=['Year_Period', 'Parent_Category'])

fig.add_trace(
        go.Scatter(x=[plot_df.Year_Period, plot_df.Parent_Category] ,y=plot_df.Inventory_Closing, name='Inventory_Closing')
)

    
fig

Demand_Domestic_Sales
Demand_Export
Supply_Cofill
Supply_Imports
Supply_Production


FigureWidget({
    'data': [{'marker': {'color': '#35193e'},
              'name': 'Demand_Domestic_Sales',
  …

In [25]:
plot_df.Category.value_counts(dropna=False)

Demand_Export            17
Supply_Cofill            17
Supply_Production        17
Demand_Domestic_Sales    17
Supply_Imports           17
Name: Category, dtype: int64

In [26]:
print(plot_df[['Year_Period', 'Parent_Category', 'Inventory_Closing']].head(20))

   Year_Period Parent_Category  Inventory_Closing
0      2020_12          Demand         21,668,934
1      2020_12          Demand         21,668,934
2      2020_12          Supply         21,668,934
3      2020_12          Supply         21,668,934
4      2020_12          Supply         21,668,934
5      2021_01          Demand         25,945,292
6      2021_01          Demand         25,945,292
7      2021_01          Supply         25,945,292
8      2021_01          Supply         25,945,292
9      2021_01          Supply         25,945,292
10     2021_02          Demand         27,712,590
11     2021_02          Demand         27,712,590
12     2021_02          Supply         27,712,590
13     2021_02          Supply         27,712,590
14     2021_02          Supply         27,712,590
15     2021_03          Demand         27,291,247
16     2021_03          Demand         27,291,247
17     2021_03          Supply         27,291,247
18     2021_03          Supply         27,291,247


In [28]:
dsi_plan['inventory_short_fall'] = round(dsi_plan['DSI_Target_National'] - dsi_plan['DSI_National'], 2)
dsi_plan['inventory_short_fall_pct'] = round((dsi_plan['short_fall'])*100/dsi_plan['DSI_National'], 2)

KeyError: 'DSI_Target_National'

In [29]:
# INVENTORY
#' horizontal


plot_df = dsi_plan[dsi_plan.Line_Portfolio == 'LPET']
index_cols = ['Year_Period']
bar_cols = ['DSI_National','DSI_North','DSI_Scotland', 'DSI_South']
line_cols = ['DSI_Target_National']
plot_df = plot_df[index_cols + bar_cols + line_cols]

fig = go.Figure()
fig = go.FigureWidget(fig)


fig.update_layout(
    template="simple_white",
    xaxis=dict(title_text="Year Period"),
    yaxis=dict(title_text="Days", tickformat=".1s")    
)

colors = sns.color_palette("Oranges_r").as_hex() #["#2A66DE","#fd5f42", "#c55f42", "#c58a42", "#c5c142", "#FFC32B"]


for column, color in zip(bar_cols, colors):       
    fig.add_trace(
        go.Bar(x=plot_df['Year_Period'], y=plot_df[column], name=column, marker_color=color),
    )

colors = sns.color_palette("rocket").as_hex()
for column, color in zip(line_cols, colors):       
    fig.add_trace(
        go.Scatter(x=plot_df['Year_Period'], y=plot_df[column], name=column, marker_color=color),
    )

    
fig

KeyError: "['DSI_South', 'DSI_National', 'DSI_Scotland', 'DSI_Target_National', 'DSI_North'] not in index"

In [30]:
# Production Line Plan

pivot_cols = ['KeyField']
pivot_rows = ['Line_Portfolio', 'Site_SAP', 'Line', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'Hours'}

df = sP_portSummarisedIHLineLoading.loc[(sP_portSummarisedIHLineLoading[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)




production_line_plan = pd.DataFrame(wide_df.to_records())

production_line_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                     for hdr in production_line_plan.columns]

#Calcuated fields
production_line_plan['Capacity_Utilisation'] = production_line_plan['Consumed']*100 / production_line_plan['Capacity']

production_line_plan = production_line_plan.fillna(0)



# Shift Data

pivot_cols = ['Hours_Type']
pivot_rows = ['Site_SAP', 'Line', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {}

# Conditional categories
conditions = [
    sP_shiftData.KeyField == 'Hours_Spare',
    sP_shiftData.KeyField == 'Hours_DownTime', 
    sP_shiftData.KeyField == 'Hours_SchedPDT',
    sP_shiftData.KeyField == 'Hours_UnCrewed',
    ]
choices = ['Spare_Hours','Planned_Downtime', 'Loss_Factor', 'Uncrewed']

sP_shiftData['Hours_Type'] = np.select(conditions, choices, default='Hours_Other')

#'Filter'
df = sP_shiftData.loc[(sP_shiftData[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

# pivot
wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)


shift_plan = pd.DataFrame(wide_df.to_records())

shift_plan.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                     for hdr in shift_plan.columns]

shift_plan = shift_plan.fillna(0)

#Available Vol

pivot_cols = ['KeyField']
pivot_rows = ['Line_Portfolio', 'Site_SAP', 'Line', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}

df = sP_portSummarisedIHLineLoading.loc[(sP_portSummarisedIHLineLoading[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)


Available_Vol = pd.DataFrame(wide_df.to_records())

Available_Vol.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                     for hdr in Available_Vol.columns]


Available_Vol['Available_Vol'] = Available_Vol['EstimatedUnconsumedCapacity']
Available_Vol['Available_Vol_with_Safety_Hours'] = Available_Vol['EstimatedUnconsumedCapacity'] + Available_Vol['EstimatedHoursSpareCapacity']

Available_Vol = Available_Vol.fillna(0)



Production_Hours = pd.merge(production_line_plan, shift_plan, on=['Site_SAP', 'Line', 'Year_Period'], how='inner')

Production_Hours = pd.merge(Production_Hours, Available_Vol, \
                            on=['Line_Portfolio', 'Site_SAP', 'Line', 'Year_Period'], \
                            how='inner')


# key_cols = ['Site_SAP', 'Line', 'Year_Period']
# value_cols = list(set(combined_df.columns) - set(key_cols))

# Production_Hours = Production_Hours.melt(id_vars=key_cols, 
#                                           value_vars=value_cols,
#                                           var_name='Category', value_name='Value')

NameError: name 'sP_shiftData' is not defined

In [ ]:
sns.color_palette("Oranges_r").as_hex()[:3] + sns.color_palette("Blues").as_hex()[:2] + sns.color_palette("Greens").as_hex()[:1]

In [ ]:
# Uncrewed in bit different COLOR
# Safety Hours

plot_df = Production_Hours[Production_Hours.Line_Portfolio == 'Aseptic']
index_cols = ['Year_Period']
bar_cols = ['Consumed','SpareHours','Capacity', 'Planned_Downtime', 'Loss_Factor', 'Uncrewed']
line_cols = []
plot_df = plot_df[index_cols + bar_cols + line_cols]

plot_df = plot_df.groupby(index_cols).sum().reset_index()

fig = go.Figure()

fig.update_layout(
    template="simple_white",
    xaxis=dict(title_text="Year Period"),
    yaxis=dict(title_text="Days", tickformat='.1s'),
    barmode="stack",
)

colors = sns.color_palette("Oranges_r").as_hex()[:3] + \
            sns.color_palette("Blues").as_hex()[:2] + \
            sns.color_palette("Greens").as_hex()[:1]


for column, color in zip(bar_cols, colors):       
    fig.add_trace(
        go.Bar(x=plot_df['Year_Period'], y=plot_df[column], name=column, marker_color=color),
    )

colors = sns.color_palette("rocket").as_hex()
for column, color in zip(line_cols, colors):       
    fig.add_trace(
        go.Scatter(x=plot_df['Year_Period'], y=plot_df[column], name=column, marker_color=color),
    )

    
fig

In [ ]:
plot_df

In [ ]:
Production_Hours.head()

In [79]:
# Capacity above Available
# Uilisation with % symbol
# number is K (1000) 
# Configurable Color Triggers for utilisation over / below X% 
# blue low, yello = high, red = very high, gree = okay

plot_df = Production_Hours[Production_Hours.Line_Portfolio == 'LPET']

key_cols = ['Site_SAP','Line', 'Year_Period']
value_cols = ['Capacity_Utilisation', 'Available_Vol']

long_df = plot_df.melt(id_vars=key_cols, 
              value_vars=value_cols,
              var_name='Line_Utilisation', value_name='Value')

long_df

pivot_cols = ['Year_Period']
pivot_rows = ['Site_SAP','Line', 'Line_Utilisation']
value_col = ['Value']
agg_func = np.sum

pd.pivot_table(long_df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

Value                                \
Year_Period                        2020_06   2020_07   2020_08   2020_09   
Site_SAP Line Line_Utilisation                                             
5502     LN03 Available_Vol         25,805   141,372   219,355   149,186   
              Capacity_Utilisation      99        96        93        96   
         LN11 Available_Vol        145,770   361,037   255,723   332,076   
              Capacity_Utilisation      97        90        92        92   
5503     LN03 Available_Vol              4         3    42,945    21,577   
              Capacity_Utilisation     100       100        95        98   
5506     LN02 Available_Vol        890,448 1,329,716 1,638,863 1,342,146   
              Capacity_Utilisation      85        73        66        74   
         LN04 Available_Vol         45,681   105,343   288,961    67,890   
              Capacity_Utilisation      97        93        82        97   

                                                                          \
Year_Period                          2020_10   2020_11 2020_12   2021_01   
Site_SAP Line Line_Utilisation                                             
5502     LN03 Available_Vol           21,865    23,344 452,727   970,675   
              Capacity_Utilisation        99        99      90        61   
         LN11 Available_Vol           46,725     6,017 608,857 1,179,845   
              Capacity_Utilisation        99       100      88        59   
5503     LN03 Available_Vol                2         3       2   256,136   
              Capacity_Utilisation       100       100     100        61   
5506     LN02 Available_Vol        1,392,757 1,334,427 883,435 2,124,237   
              Capacity_Utilisation        71        72      84        42   
         LN04 Available_Vol           83,261    40,323 342,390   625,167   
              Capacity_Utilisation        93        97      84        49   

                                                                          
Year_Period                          2021_02   2021_03   2021_04 2021_05  
Site_SAP Line Line_Utilisation                                            
5502     LN03 Available_Vol          780,555 2,037,117 1,702,802     nan  
              Capacity_Utilisation        77        65        55     nan  
         LN11 Available_Vol          859,300 2,350,923 2,666,419 899,357  
              Capacity_Utilisation        78        60        44       0  
5503     LN03 Available_Vol          179,831 2,010,798 1,792,256 489,082  
              Capacity_Utilisation        79        31        24       0  
5506     LN02 Available_Vol        2,640,915 3,702,013 2,216,865     nan  
              Capacity_Utilisation        50        48        48     nan  
         LN04 Available_Vol          455,704   792,644   532,593     nan  
              Capacity_Utilisation        72        66        63     nan

In [18]:
x = pd.pivot_table(long_df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)
x.columns = x.columns.to_series().str.join('_')

x.reset_index()

,Site_SAP,Line,Line_Utilisation,Value_2020_06,Value_2020_07,Value_2020_08,Value_2020_09,Value_2020_10,Value_2020_11,Value_2020_12,Value_2021_01,Value_2021_02,Value_2021_03,Value_2021_04,Value_2021_05,Value_2021_06,Value_2021_07
0,5501,LN03,Available_Vol_with_Safety_Hours,"31,075","25,029","24,590","425,586","495,514","593,552","391,085","426,417","594,432","1,245,076","942,934","1,035,188","1,337,052","1,015,662"
1,5501,LN03,Capacity_Utilisation,100,100,100,59,37,24,49,28,24,22,26,18,14,0


In [4]:
# LINE Utilisation

#Available Vol

pivot_cols = ['KeyField']
pivot_rows = ['Line_Portfolio', 'Site_SAP', 'Line', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}

df = sP_portSummarisedIHLineLoading.loc[(sP_portSummarisedIHLineLoading[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)


Available_Vol = pd.DataFrame(wide_df.to_records())

Available_Vol.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                     for hdr in Available_Vol.columns]


Available_Vol['Available_Vol'] = Available_Vol['EstimatedUnconsumedCapacity']
Available_Vol['Available_Vol_with_Safety_Hours'] = Available_Vol['EstimatedUnconsumedCapacity'] + Available_Vol['EstimatedHoursSpareCapacity']



In [5]:
Available_Vol.Year_Period.value_counts(dropna=False)

2022_04    62
2021_10    61
2020_12    61
2021_11    61
2021_05    60
2021_12    60
2021_04    60
2021_07    60
2022_03    60
2021_09    60
2022_02    60
2021_01    60
2022_01    60
2021_06    60
2021_03    60
2021_02    59
2021_08    59
Name: Year_Period, dtype: int64

In [20]:
#Sourcing

pivot_cols = ['KeyField']
pivot_rows = ['Level2Region', 'Summary_Portfolio', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}


#DEMAND

# Domestic Sales

df = dP_siteSummarised.loc[(dP_siteSummarised[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                    columns=pivot_cols, aggfunc=agg_func)

domestic_sales = pd.DataFrame(wide_df.to_records())

domestic_sales.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                     for hdr in domestic_sales.columns]


domestic_sales = domestic_sales.fillna(0)
domestic_sales.columns = ['Summary_Portfolio', 'Level2Region', 'Year_Period', 'Domestic_Sales', 'Total']




In [29]:
#IBT Out

pivot_cols = ['IBT_Out' , 'IBT_In']
pivot_rows = ['Summary_Portfolio', 'From_GeoRegion', 'To_GeoRegion', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC', 'ToCountry':'GB', 'ToRepack_Flag':False}

mP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'mP_Summarised'))

# Conditional categories
conditions = [
    (mP_Summarised.From_GeoRegion == 'South') &  (mP_Summarised.To_GeoRegion != 'South'),
    (mP_Summarised.From_GeoRegion == 'North') &  (mP_Summarised.To_GeoRegion != 'North'),
    (mP_Summarised.From_GeoRegion == 'Scotland') &  (mP_Summarised.To_GeoRegion != 'Scotland')    
    ]
choices = ['IBT_Out_South','IBT_Out_North', 'IBT_Out_Scotland']

mP_Summarised['IBT_Out'] = np.select(conditions, choices, default='IBT_Out_Other')

# Conditional categories
conditions = [    
    (mP_Summarised.From_GeoRegion != 'South') &  (mP_Summarised.To_GeoRegion == 'South'),
    (mP_Summarised.From_GeoRegion != 'North') &  (mP_Summarised.To_GeoRegion == 'North'),
    (mP_Summarised.From_GeoRegion != 'Scotland') &  (mP_Summarised.To_GeoRegion == 'Scotland'),
    ]
choices = ['IBT_In_South','IBT_In_North', 'IBT_In_Scotland']

mP_Summarised['IBT_In'] = np.select(conditions, choices, default='IBT_Other')


ibt_out = mP_Summarised.loc[(mP_Summarised[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

ibt_out.head()

# wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
#                     columns=pivot_cols, aggfunc=agg_func)

# wide_df.fillna(0)


,FromCountry,ToCountry,SiteCountry,From_GeoRegion,To_GeoRegion,Site_SAP_From,Site_SAP_To,Year_Period,Summary_Portfolio,KeyField,Value,Uom,ToRepack_Flag,IBT_Out,IBT_In
4,NaN,GB,nan,NaN,North,DIS_NL,5502,2020_06,Monster,ZUC,"699,935",ZUC,False,IBT_Out_Other,IBT_In_North
9,NaN,GB,nan,NaN,North,DIS_NL,5502,2020_07,Monster,ZUC,"289,764",ZUC,False,IBT_Out_Other,IBT_In_North
14,NaN,GB,nan,NaN,North,DIS_NL,5502,2020_08,Monster,ZUC,"280,065",ZUC,False,IBT_Out_Other,IBT_In_North
19,NaN,GB,nan,NaN,North,DIS_NL,5502,2020_09,Monster,ZUC,"342,749",ZUC,False,IBT_Out_Other,IBT_In_North
24,NaN,GB,nan,NaN,North,DIS_NL,5502,2020_10,Monster,ZUC,"135,180",ZUC,False,IBT_Out_Other,IBT_In_North


In [22]:
#IBT Out

pivot_cols = ['KeyField']
pivot_rows = ['Summary_Portfolio', 'From_GeoRegion', 'To_GeoRegion', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC', 'ToCountry':'GB', 'ToRepack_Flag':False}

mP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'mP_Summarised'))

df = mP_Summarised.loc[(mP_Summarised[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                     columns=pivot_cols, aggfunc=agg_func)

ibt_df = pd.DataFrame(wide_df.to_records())

ibt_df.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                     for hdr in ibt_df.columns]




In [23]:
# Imports and Export


pivot_cols = ['Import_Export']
pivot_rows = ['Summary_Portfolio', 'To_GeoRegion', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}


mP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'mP_Summarised'))

df = mP_Summarised.copy()
df.To_GeoRegion = df.To_GeoRegion.fillna('NA')

# Conditional categories
conditions = [
    (df.FromCountry == 'GB') &  (df.ToCountry != 'GB'),
    (df.FromCountry != 'GB') &  (df.ToCountry == 'GB'),
    (df.FromCountry == 'GB') &  (df.ToCountry == 'GB')
    ]
choices = ['Export', 'Import', 'IBT', ]

df['Import_Export'] = np.select(conditions, choices, default='UNKNOWN')

df = df.loc[(df[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]


wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                     columns=pivot_cols, aggfunc=agg_func)

imp_exp_df = pd.DataFrame(wide_df.to_records())

imp_exp_df.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                     for hdr in imp_exp_df.columns]


imp_exp_df = imp_exp_df.fillna(0)


In [24]:
#Repack

pivot_cols = ['Uom']
pivot_rows = ['Summary_Portfolio', 'From_GeoRegion', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC', 'FromCountry':'GB', 'ToRepack_Flag':False}

mP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'mP_Summarised'))

df = mP_Summarised.copy()
df[pivot_rows] = df[pivot_rows].fillna('NA')


df = df.loc[(df[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

df

wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                     columns=pivot_cols, aggfunc=agg_func)

repack_df = pd.DataFrame(wide_df.to_records())

repack_df.columns = [hdr.replace("('Value', '", "Repack").replace("')", "") \
                     for hdr in repack_df.columns]


repack_df = repack_df.fillna(0)


In [25]:
#Cofill

pivot_cols = ['Uom']
pivot_rows = ['Summary_Portfolio', 'To_GeoRegion', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'ZUC'}

mP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'mP_Summarised'))

df = mP_Summarised.copy()
df[pivot_rows] = df[pivot_rows].fillna('NA')
df = df.loc[df.Site_SAP_From.str.contains('_GB')]


df = df.loc[(df[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

df

wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                     columns=pivot_cols, aggfunc=agg_func)

cofill_df = pd.DataFrame(wide_df.to_records())

cofill_df.columns = [hdr.replace("('Value', '", "Cofill").replace("')", "") \
                     for hdr in cofill_df.columns]


cofill_df = cofill_df.fillna(0)

In [26]:
domestic_sales.head()

,Summary_Portfolio,Level2Region,Year_Period,Domestic_Sales,Total
0,North,Aseptic,2020_06,"88,771","88,771"
1,North,Aseptic,2020_07,"94,372","94,372"
2,North,Aseptic,2020_08,"98,067","98,067"
3,North,Aseptic,2020_09,"117,586","117,586"
4,North,Aseptic,2020_10,"85,737","85,737"


In [27]:
ibt_df.head()

,Summary_Portfolio,From_GeoRegion,To_GeoRegion,Year_Period,ZUC
0,Aseptic,South,North,2020_06,"21,878"
1,Aseptic,South,North,2020_07,"21,191"
2,Aseptic,South,North,2020_08,"20,345"
3,Aseptic,South,North,2020_09,"18,849"
4,Aseptic,South,North,2020_10,"13,099"


In [28]:
imp_exp_df.head()

,Summary_Portfolio,To_GeoRegion,Year_Period,Export,IBT,Import
0,Aseptic,NA,2020_06,"1,042,555",0,0
1,Aseptic,NA,2020_07,"801,060",0,0
2,Aseptic,NA,2020_08,"769,479",0,0
3,Aseptic,NA,2020_09,"499,982",0,0
4,Aseptic,NA,2020_10,"303,249",0,0


In [80]:
#Freight 
# Highlight TOTAL Rows

#pd.options.display.float_format = '{:,.0f}'.format

pivot_cols = ['Year_Period']
pivot_rows = ['Summary_Portfolio', 'From_GeoRegion', 'To_GeoRegion']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'Trucks', 'Summary_Portfolio':'LPET'}

mP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'mP_Summarised'))

df = mP_Summarised.copy()
df = df.loc[ ~df.To_GeoRegion.isna() & ~df.From_GeoRegion.isna() ]
df[pivot_rows] = df[pivot_rows].fillna('NA')
df[value_col] = df[value_col].fillna(0)

df['From_GeoRegion'] = pd.Categorical(df['From_GeoRegion'], ["North", "South", "Scotland"])
df['To_GeoRegion'] = pd.Categorical(df['To_GeoRegion'], ["North", "South", "Scotland"])


df = df.loc[(df[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]


wide_df = pd.pivot_table(df, values=value_col, index=pivot_rows,
                     columns=pivot_cols, aggfunc=agg_func)

wide_df = wide_df.fillna(0)

wide_df = wide_df.round(0).astype(int)

cm = sns.light_palette("pink", as_cmap=True)
wide_df.style.background_gradient(cmap=cm)

# freight_df = pd.DataFrame(wide_df.to_records())

# freight_df.columns = [hdr.replace("('Value', '", "").replace("')", "") \
#                      for hdr in freight_df.columns]


# freight_df = freight_df.fillna(0)


In [28]:
wide_df

pd.concat([
    d.append(d.sum().rename((k, 'Total')))
    for k, d in wide_df.groupby(level=1)
]).append(wide_df.sum().rename(('Grand', 'Total')))



Value                                          \
Year_Period             2020_06 2020_07 2020_08 2020_09 2020_10 2020_11   
(LPET, North, North)        206     158     153     180     131     163   
(LPET, North, South)        183      36      34      47      61      64   
(LPET, North, Scotland)      66      35      26      39      39      37   
(North, Total)              455     229     213     266     231     264   
(LPET, South, North)        287     141     146     176     124     144   
(LPET, South, South)        168     147     133     162     128     134   
(LPET, South, Scotland)      42      22      17      23      19      17   
(South, Total)              497     310     296     361     271     295   
(LPET, Scotland, North)     100      72      45      37      89      43   
(LPET, Scotland, South)      36       3       5      11      22       6   
(Scotland, Total)           136      75      50      48     111      49   
(Grand, Total)             1088     614     559     675     613     608   

                                                                         
Year_Period             2020_12 2021_01 2021_02 2021_03 2021_04 2021_05  
(LPET, North, North)        175      96     138     170      48       0  
(LPET, North, South)         45      12      23      32      20       0  
(LPET, North, Scotland)      49      18      30      36      19       0  
(North, Total)              269     126     191     238      87       0  
(LPET, South, North)        200      88     150     199     117       0  
(LPET, South, South)        185      70     127     168      98       0  
(LPET, South, Scotland)      18       4       5       6       4       0  
(South, Total)              403     162     282     373     219       0  
(LPET, Scotland, North)      80      17      31      42      25       0  
(LPET, Scotland, South)      50       1       3       4       3       0  
(Scotland, Total)           130      18      34      46      28       0  
(Grand, Total)              802     306     507     657     334       0

In [128]:
# DSI Heatmap

# Configure Trigger

pivot_cols = ['Year_Period']
pivot_rows = ['Summary_Portfolio', 'DSI_Type']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom':'Days'}

ip_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'iP_Summarised'))

df = ip_Summarised.copy()

df = df[df.KeyField.str.contains('Days_ClosingStockTarget_National_SummaryPortfolio|Days_DSI_National_SummaryPortfolio')]

# Conditional categories
conditions = [
    (df.KeyField  == 'Days_ClosingStockTarget_National_SummaryPortfolio'),
    (df.KeyField  == 'Days_DSI_National_SummaryPortfolio'),
    ]
choices = ['Target_DSI', 'Planned_DSI']

df['DSI_Type'] = np.select(conditions, choices, default='Other_DSI')

df = df.loc[(df[list(filter_by)] == pd.Series(filter_by)).all(axis=1)]

#Create Pivot with Percentage Difference

wide_df = pd.pivot_table(df, values=value_col, index=['Summary_Portfolio', 'Year_Period'],
                     columns=['DSI_Type'], aggfunc=agg_func)
wide_df = pd.DataFrame(wide_df.to_records())
wide_df.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in wide_df.columns]

wide_df['Percent_Diff'] = (wide_df['Planned_DSI'] - wide_df['Target_DSI'])*100/wide_df['Target_DSI']

wide_df = pd.melt(wide_df, id_vars=['Summary_Portfolio','Year_Period'], 
        value_vars=['Planned_DSI','Target_DSI','Percent_Diff'], 
        var_name='DSI_Type',
        value_name='Value')

wide_df['DSI_Type'] = pd.Categorical(wide_df['DSI_Type'], ["Target_DSI", "Planned_DSI", "Percent_Diff"])

wide_df = pd.pivot(wide_df, values=['Value'], index=['Summary_Portfolio', 'DSI_Type'], columns=['Year_Period'])

wide_df = wide_df.fillna(0)

wide_df = pd.DataFrame(wide_df.to_records())

wide_df.columns = [hdr.replace("('Value', '", "").replace("')", "") \
                     for hdr in wide_df.columns]

wide_df

,Summary_Portfolio,DSI_Type,2020_06,2020_07,2020_08,2020_09,2020_10,2020_11,2020_12,2021_01,2021_02,2021_03,2021_04,2021_05,2021_06,2021_07,2021_08
0,Aseptic,Target_DSI,35,34,34,34,34,36,29,34,34,34,28,0,0,0,0
1,Aseptic,Planned_DSI,33,31,32,50,37,37,31,37,36,36,37,0,0,0,0
2,Aseptic,Percent_Diff,-5,-10,-8,48,10,3,8,8,7,7,32,0,0,0,0
3,Cans - Board,Target_DSI,16,15,15,15,15,15,14,15,15,15,28,0,0,0,0
4,Cans - Board,Planned_DSI,16,15,15,15,15,16,14,15,15,15,28,0,0,0,0
5,Cans - Board,Percent_Diff,0,0,0,0,5,12,-0,-0,0,0,0,0,0,0,0
6,Cans - Shrink,Target_DSI,18,18,18,18,17,18,19,17,17,18,28,0,0,0,0
7,Cans - Shrink,Planned_DSI,18,18,30,27,21,24,23,25,18,18,28,0,0,0,0
8,Cans - Shrink,Percent_Diff,0,0,70,54,23,31,22,50,2,0,0,0,0,0,0
9,Cans - Singles,Target_DSI,15,16,15,15,15,15,17,14,15,15,28,0,0,0,0


In [62]:
INPUT_PICKLE_DIR = '../input/pickle'

iP_Summarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'iP_Summarised')).round(2)

df = iP_Summarised[iP_Summarised.KeyField.isin(['ZUC_ClosingStockTarget', 'ZUC_StockVsTarget'])]

inventory_shortfalls = pd.pivot_table(df, values=value_col, index=pivot_rows, columns=pivot_cols, aggfunc=agg_func)

inventory_shortfalls = pd.DataFrame(inventory_shortfalls.to_records())

inventory_shortfalls.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in inventory_shortfalls.columns]
inventory_shortfalls
#inventory_shortfalls['ZUC_StockVsTarget'] = inventory_shortfalls['ZUC_StockVsTarget'].astype(float)

,Summary_Portfolio,Year_Period,ZUC
0,Aseptic,2020_12,1077127914461293489163022590010822906353817071...
1,Aseptic,2021_01,1093133715050001769244700793227629010021219713...
2,Aseptic,2021_02,1176144115940001999280300104727463353000251148...
3,Aseptic,2021_03,1631175918420002417354000101728313460000378845...
4,Aseptic,2021_04,1700184819190002593381600904248630600004114516...
...,...,...,...
289,Water,2021_12,1105619022020400005089752898073056494121000005...
290,Water,2022_01,1266401028230390000580385841118334853107326000...
291,Water,2022_02,1447341180264370001030665898501283239994123158...
292,Water,2022_03,152774124327860-100070171038013523421461297840...


In [69]:
dP_siteSummarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'dP_siteSummarised')).round(2)
dP_siteSummarised['Value'] = dP_siteSummarised['Value'].fillna(0)
dP_siteSummarised['Value'] = dP_siteSummarised['Value'].astype(float)
#dP_siteSummarised

ValueError: could not convert string to float: 

In [80]:
def FloatOrZero(value):
    try:
        return float(value)
    except:
        return 0.0

In [81]:
dP_siteSummarised['Value'].apply(FloatOrZero)

0          949
1        1,003
2          949
3        1,003
4        1,124
          ... 
281399   5,582
281400   2,488
281401   2,629
281402   2,488
281403   2,629
Name: Value, Length: 281404, dtype: float64

In [74]:
df_obj = dP_siteSummarised.select_dtypes(['object'])
df_obj

,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,Material,KeyField,Value,Uom
0,North,5502,2020_12,Aseptic,203380,Available,949.192017555237,ZRW
1,North,5502,2020_12,Aseptic,203380,Available,1003,ZUC
2,North,5502,2020_12,Aseptic,203380,Total,949.192017555237,ZRW
3,North,5502,2020_12,Aseptic,203380,Total,1003,ZUC
4,North,5502,2020_12,Aseptic,203381,Available,1124.26731491089,ZRW
...,...,...,...,...,...,...,...,...
281399,South,5559,2022_04,Water,466485,Total,5582,ZUC
281400,South,5559,2022_04,Water,466486,Available,2487.96192836761,ZRW
281401,South,5559,2022_04,Water,466486,Available,2629,ZUC
281402,South,5559,2022_04,Water,466486,Total,2487.96192836761,ZRW


In [102]:
len('')

0

In [101]:
shift_lever = pd.read_csv('../input/levers/shift_lever.csv')
shift_lever = shift_lever['Line'].str.strip()
shift_lever

0      LN01
1      LN01
2      LN01
3      LN01
4      LN01
       ... 
491    LN01
492    LN01
493    LN01
494    LN01
495    LN01
Name: Line, Length: 496, dtype: object

In [87]:
production_capability = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'production_capacity'))
production_capability = production_capability[['Plant', 'Line', 'Material']].drop_duplicates()
production_capability.columns = ['Site_SAP', 'Line', 'Material']
production_capability.Site_SAP = production_capability.Site_SAP.astype(str)

In [89]:
shift_lever = pd.merge(shift_lever, production_capability, on=['Site_SAP', 'Line'])

In [90]:
shift_lever.dtypes

Site_SAP                             object
Line                                 object
Year_Period                          object
Weeks_In_Period                       int64
Shift_Length                        float64
Shift_Days_In_Week                    int64
Hours_Overtime                        int64
Hours_Crew                          float64
Hours_UnCrewed                      float64
Loss_Factor                         float64
Hours_Down_Time                       int64
Hours_Sched_PDT                     float64
Safety_Hours                        float64
Hours_Total_Available                 int64
Hours_Total_Available_Calc          float64
Additional_Hours                    float64
Additional_Hours_Used               float64
Material_Site_DemandShortfall       float64
Material_Site_InventoryShortfall    float64
Material                             object
dtype: object

In [93]:
len(shift_lever.Material.unique())

1923

In [106]:
SERVER_NAME

NameError: name 'SERVER_NAME' is not defined

In [ ]:
conn = pyodbc.connect('Driver={SQL Server};'
                            f'Server={SERVER_NAME};'
                            f'Database={DATABASE_NAME};'
                            'Trusted_Connection=yes;')
      

    sql = f'''
            WITH DSI_TABLE AS (
                SELECT	  
                    CASE PD.[Site_SAP]
                            WHEN 'P67' THEN '5567'
                            WHEN 'P69' THEN '5569'
                            WHEN 'P14' THEN '5504'
                            ELSE PD.[Site_SAP]
                        END AS Site_SAP
                    ,CASE PD.[Site_SAP]
                            WHEN 'P67' THEN 'North'
                            WHEN 'P69' THEN 'North'
                            WHEN 'P14' THEN 'South'
                            ELSE PD.[Level2Region]
                        END AS Level2Region
                    ,PD.[Summary_Portfolio] 
                    ,PD.[Material]
                    ,CONCAT([Year],'_',format (CAST(PD.[CCEP_Period] AS INT), '0#')) AS Year_Period
                    ,CASE CAST(PD.[CCEP_Period] AS INT) % 3 WHEN 0 THEN 5 ELSE 4 END AS BucketsInPeriod
                    ,ROUND(convert(float,PD.[Value]),0) AS TotalDemand
                    ,ROUND(ROUND(convert(float,PD.[Value]),0) / ((CASE CAST(PD.[CCEP_Period] AS INT) % 3 WHEN 0 THEN 5 ELSE 4 END)*7),2) AS PeriodDailyDemand
                    ,ROUND(convert(float, IP.[Value]),0) AS ClosingStock
                    ,ROUND(convert(float,IP_target.[Value]),0) AS ClosingStock_Target
                FROM [PACSQL_MTP].{config.PROJECTED_DEMAND_TABLE} PD
                INNER JOIN [PACSQL_MTP].{config.INVENTORY_TABLE} IP
                    ON IP.Year_Period = CONCAT(PD.[Year],'_',format (CAST(PD.[CCEP_Period] AS INT), '0#'))
                    AND IP.Material = PD.[Material]
                    AND IP.Site_SAP = PD.Site_SAP
                INNER JOIN [PACSQL_MTP].{config.INVENTORY_TABLE} IP_target
                    ON IP_target.Year_Period = CONCAT(PD.[Year],'_',format (CAST(PD.[CCEP_Period] AS INT), '0#'))
                    AND IP_target.Material = PD.[Material]
                    AND IP_target.Site_SAP = PD.Site_SAP
                WHERE 1=1
                    AND PD.[Table] = 'Demand_Site_Material'
                    AND PD.[KeyField] = '{uom}_TotalDemand'
                    AND IP.[KeyField] = '{uom}_ClosingStock'
                    AND IP_target.[KeyField] = '{uom}_ClosingStockTarget'
            ),
            DSI_GROUPED AS (
                select Summary_Portfolio, Year_Period 
                        ,SUM(TotalDemand) AS Total_Demand 
                        ,SUM(PeriodDailyDemand) AS PeriodDailyDemand 
                        ,SUM(ClosingStock) AS ClosingStock
                        ,SUM(ClosingStock_Target) AS ClosingStock_Target
                        ,ROUND(SUM(ClosingStock) / SUM(PeriodDailyDemand), 2) AS DSI_Planned
                        ,ROUND(SUM(ClosingStock_Target) / SUM(PeriodDailyDemand), 2) AS DSI_Target			
                FROM DSI_TABLE
                GROUP BY Summary_Portfolio, Year_Period
            )
            SELECT Summary_Portfolio
            ,Year_Period
            ,Total_Demand
            ,PeriodDailyDemand
            ,ClosingStock
            ,ClosingStock_Target
            ,DSI_Planned
            ,DSI_Target
            ,ROUND((DSI_Target - DSI_Planned) *100 / DSI_Target, 2) AS DSI_Under_Target_Percent
            FROM DSI_GROUPED    
    '''

    data = pd.read_sql(sql,conn) 

    planned_dsi = pd.pivot(data, values=['DSI_Planned'], index=['Summary_Portfolio'], columns=['Year_Period']).reset_index()

In [109]:
INPUT_PICKLE_DIR

'../input/pickle'

In [110]:
DSI_PICKLE = 'dsi_pickle'

In [115]:
x = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, (DSI_PICKLE + 'ZRW')))

In [116]:
y = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, (DSI_PICKLE + 'ZUC')))

In [124]:
z = pd.merge(x, y, on=['Site_SAP', 'Level2Region', 'Summary_Portfolio', 
                       'Material', 'Year_Period', 'BucketsInPeriod'], how='inner')

z

,Site_SAP,Level2Region,Summary_Portfolio,Material,Year_Period,BucketsInPeriod,ZRW_TotalDemand,ZRW_PeriodDailyDemand,ZRW_ClosingStock,ZRW_ClosingStock_Target,ZUC_TotalDemand,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target
0,5567,North,Cans - Shrink,466173,2020_12,5,521.927970409393,14.91,87.4659510850906,87.4659510850906,728,20.80,122,122
1,5567,North,Cans - Singles,220599,2020_12,5,8145.08746129274,232.72,455.253106057644,455.253106057644,11361,324.60,635,635
2,5567,North,Cans - Singles,261338,2020_12,5,28.6773610115051,0.82,0.716934025287628,0.716934025287628,40,1.14,1,1
3,5567,North,Cans - Singles,407468,2020_12,5,1758.63916403055,50.25,124.02958637476,124.02958637476,2453,70.09,173,173
4,5567,North,Cans - Singles,409815,2020_12,5,329.072717607021,9.40,51.6192498207092,51.6192498207092,459,13.11,72,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66913,5569,North,REPACK,240776,2022_03,5,2194.59251117706,62.70,1716.68426704407,1716.68426704407,2319,66.26,1814,1814
66914,5503,Scotland,Aseptic,203381,2022_03,5,238.480945587158,6.81,250.783534049988,250.783534049988,252,7.20,265,265
66915,5503,Scotland,Cans - Board,439418,2022_03,5,4933.653393507,140.96,2744.9971101284,2744.9971101284,8602,245.77,4786,4786
66916,5503,Scotland,Cans - Shrink,208269,2022_03,5,124.746520400047,3.56,63.8071282505989,63.8071282505989,174,4.97,89,89


In [120]:
z.dtypes

Site_SAP                    object
Level2Region                object
Summary_Portfolio           object
Material                    object
Year_Period                 object
BucketsInPeriod              int64
ZRW_TotalDemand             object
ZRW_PeriodDailyDemand      float64
ZRW_ClosingStock            object
ZRW_ClosingStock_Target     object
ZUC_TotalDemand             object
ZUC_PeriodDailyDemand      float64
ZUC_ClosingStock            object
ZUC_ClosingStock_Target     object
dtype: object

In [129]:
z.fillna(0)
z['ZRW_ClosingStock'] = z['ZRW_ClosingStock'].astype(float)
z = z.round()
x

,Site_SAP,Level2Region,Summary_Portfolio,Material,Year_Period,BucketsInPeriod,ZRW_TotalDemand,ZRW_PeriodDailyDemand,ZRW_ClosingStock,ZRW_ClosingStock_Target
0,5567,North,Cans - Shrink,466173,2020_12,5,521.927970409393,14.91,87.4659510850906,87.4659510850906
1,5567,North,Cans - Singles,220599,2020_12,5,8145.08746129274,232.72,455.253106057644,455.253106057644
2,5567,North,Cans - Singles,261338,2020_12,5,28.6773610115051,0.82,0.716934025287628,0.716934025287628
3,5567,North,Cans - Singles,407468,2020_12,5,1758.63916403055,50.25,124.02958637476,124.02958637476
4,5567,North,Cans - Singles,409815,2020_12,5,329.072717607021,9.40,51.6192498207092,51.6192498207092
...,...,...,...,...,...,...,...,...,...,...
66913,5569,North,REPACK,240776,2022_03,5,2194.59251117706,62.70,1716.68426704407,1716.68426704407
66914,5503,Scotland,Aseptic,203381,2022_03,5,238.480945587158,6.81,250.783534049988,250.783534049988
66915,5503,Scotland,Cans - Board,439418,2022_03,5,4933.653393507,140.96,2744.9971101284,2744.9971101284
66916,5503,Scotland,Cans - Shrink,208269,2022_03,5,124.746520400047,3.56,63.8071282505989,63.8071282505989


In [130]:
z

,Site_SAP,Level2Region,Summary_Portfolio,Material,Year_Period,BucketsInPeriod,ZRW_TotalDemand,ZRW_PeriodDailyDemand,ZRW_ClosingStock,ZRW_ClosingStock_Target,ZUC_TotalDemand,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target
0,5567,North,Cans - Shrink,466173,2020_12,5,521.927970409393,15.0,87.0,87.4659510850906,728,21.0,122,122
1,5567,North,Cans - Singles,220599,2020_12,5,8145.08746129274,233.0,455.0,455.253106057644,11361,325.0,635,635
2,5567,North,Cans - Singles,261338,2020_12,5,28.6773610115051,1.0,1.0,0.716934025287628,40,1.0,1,1
3,5567,North,Cans - Singles,407468,2020_12,5,1758.63916403055,50.0,124.0,124.02958637476,2453,70.0,173,173
4,5567,North,Cans - Singles,409815,2020_12,5,329.072717607021,9.0,52.0,51.6192498207092,459,13.0,72,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66913,5569,North,REPACK,240776,2022_03,5,2194.59251117706,63.0,1717.0,1716.68426704407,2319,66.0,1814,1814
66914,5503,Scotland,Aseptic,203381,2022_03,5,238.480945587158,7.0,251.0,250.783534049988,252,7.0,265,265
66915,5503,Scotland,Cans - Board,439418,2022_03,5,4933.653393507,141.0,2745.0,2744.9971101284,8602,246.0,4786,4786
66916,5503,Scotland,Cans - Shrink,208269,2022_03,5,124.746520400047,4.0,64.0,63.8071282505989,174,5.0,89,89


In [131]:
z.loc[z.Year_Period.isin(['2020_12'])]

,Site_SAP,Level2Region,Summary_Portfolio,Material,Year_Period,BucketsInPeriod,ZRW_TotalDemand,ZRW_PeriodDailyDemand,ZRW_ClosingStock,ZRW_ClosingStock_Target,ZUC_TotalDemand,ZUC_PeriodDailyDemand,ZUC_ClosingStock,ZUC_ClosingStock_Target
0,5567,North,Cans - Shrink,466173,2020_12,5,521.927970409393,15.0,87.0,87.4659510850906,728,21.0,122,122
1,5567,North,Cans - Singles,220599,2020_12,5,8145.08746129274,233.0,455.0,455.253106057644,11361,325.0,635,635
2,5567,North,Cans - Singles,261338,2020_12,5,28.6773610115051,1.0,1.0,0.716934025287628,40,1.0,1,1
3,5567,North,Cans - Singles,407468,2020_12,5,1758.63916403055,50.0,124.0,124.02958637476,2453,70.0,173,173
4,5567,North,Cans - Singles,409815,2020_12,5,329.072717607021,9.0,52.0,51.6192498207092,459,13.0,72,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11473,5559,South,GB,270074,2020_12,5,211196.643435478,6034.0,26536.0,26535.7369613647,446338,12753.0,56080,56080
11474,5559,South,GB,272182,2020_12,5,58726.3356565833,1678.0,10934.0,10934.4319546223,108597,3103.0,20220,20220
11475,5559,South,GB,272768,2020_12,5,1062.0783560276,30.0,29.0,28.6609739661217,1964,56.0,53,53
11476,5559,South,GB,272791,2020_12,5,106282.061730385,3037.0,17771.0,17770.615858078,224614,6418.0,37556,37556


In [ ]:
SUPPLY_PLAN_INHOUSE_LINELOADING_PICKLE = sP_portSummarisedIHLineLoading

In [138]:
df = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, SUPPLY_PLAN_INHOUSE_LINELOADING_PICKLE))
df

,Line_Portfolio,Material,Site_SAP,Line,Year_Period,KeyField,Value,Uom
0,Aseptic,211313,5501,LN03,2020_12,Planned,11963.79,ZRW
1,Aseptic,211313,5501,LN03,2020_12,Planned,12642.00,ZUC
2,Aseptic,211313,5501,LN03,2021_01,Planned,17862.41,ZRW
3,Aseptic,211313,5501,LN03,2021_01,Planned,18875.00,ZUC
4,Aseptic,211313,5501,LN03,2021_02,Planned,23326.65,ZRW
...,...,...,...,...,...,...,...,...
31690,Water,,5546,LN01,2021_12,SpareHours,0.00,Hours
31691,Water,,5546,LN01,2022_01,SpareHours,0.00,Hours
31692,Water,,5546,LN01,2022_02,SpareHours,0.00,Hours
31693,Water,,5546,LN01,2022_03,SpareHours,0.00,Hours


In [139]:
df = summarise_df_for_visualisation(df)
df

,Line_Portfolio,Site_SAP,Line,Year_Period,KeyField,Uom,Value
0,Aseptic,5501,LN03,2020_12,Capacity,Hours,332.0
1,Aseptic,5501,LN03,2020_12,Consumed,Hours,20.0
2,Aseptic,5501,LN03,2020_12,EstimatedHoursSpareCapacity,ZRW,23728.0
3,Aseptic,5501,LN03,2020_12,EstimatedHoursSpareCapacity,ZUC,22529.0
4,Aseptic,5501,LN03,2020_12,EstimatedUnconsumedCapacity,ZRW,410282.0
...,...,...,...,...,...,...,...
5550,Water,5546,LN01,2022_04,EstimatedUnconsumedCapacity,ZRW,421331.0
5551,Water,5546,LN01,2022_04,EstimatedUnconsumedCapacity,ZUC,925122.0
5552,Water,5546,LN01,2022_04,Planned,ZRW,401864.0
5553,Water,5546,LN01,2022_04,Planned,ZUC,888518.0


In [183]:
def get_next_period(this_period) :
    this_month = int(this_period.split('_')[1])
    this_year = int(this_period.split('_')[0])        
    
    next_year = this_year
    if this_month==12:
        next_year = this_year+1
        next_month = 1
    else:
        next_month = this_month+1
   
    return f"{next_year}_{next_month:02d}"

def get_inventory_excess():
    # Inventory Shortfalls
    pivot_cols = ['KeyField']
    pivot_rows = ['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period']
    value_col = ['Value']
    agg_func = np.sum
    filter_by = {'Uom': 'ZUC'}

    iP_Summarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'iP_Summarised')).round(2)
    iP_Summarised['Value'] = iP_Summarised['Value'].astype(float)
    
    df = iP_Summarised[iP_Summarised.KeyField.isin(['ZUC_StockVsTarget'])]

    inventory_excess = pd.pivot_table(df, values=value_col, index=pivot_rows, columns=pivot_cols, aggfunc=agg_func)

    inventory_excess = pd.DataFrame(inventory_excess.to_records())

    inventory_excess.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in inventory_excess.columns]

    num = inventory_excess._get_numeric_data()
    num[num < 0] = 0

    # shift period by 1 as excess inventory in this period will be supply in next period
    inventory_excess.Year_Period = [ get_next_period(x) for x in  inventory_excess.Year_Period ]

    #inventory_excess.columns = [pivot_rows + ['Excess_Inventory']]
    
    inventory_excess.rename(columns={'ZUC_StockVsTarget':'Excess_Inventory'}, inplace=True)

    return inventory_excess

In [184]:
excess_inventory = get_inventory_excess()
excess_inventory.head()

,Site_SAP,Summary_Portfolio,Material,Year_Period,Excess_Inventory
0,5501,Aseptic,203380,2021_01,385.0
1,5501,Aseptic,203380,2021_02,0.0
2,5501,Aseptic,203380,2021_03,0.0
3,5501,Aseptic,203380,2021_04,0.0
4,5501,Aseptic,203380,2021_05,0.0


In [155]:
dP_siteSummarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'dP_siteSummarised')).round(2)
dP_siteSummarised['Value'] = dP_siteSummarised['Value'].astype(float)

# Demand Shortfalls
demand_plan = dP_siteSummarised.loc[dP_siteSummarised.Uom == 'ZUC'].pivot(index=['Level2Region','Site_SAP','Year_Period','Summary_Portfolio', 'Material', 'Uom'], \
                                    columns='KeyField')['Value'].reset_index()


demand_plan.head()

KeyField,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,Material,Uom,Available,Total
0,North,5502,2020_12,Aseptic,203380,ZUC,1003.0,1003.0
1,North,5502,2020_12,Aseptic,203381,ZUC,1188.0,1188.0
2,North,5502,2020_12,Aseptic,203475,ZUC,1192.0,1192.0
3,North,5502,2020_12,Aseptic,211313,ZUC,4112.0,4112.0
4,North,5502,2020_12,Aseptic,214633,ZUC,1565.0,1565.0


In [152]:
demand_plan[['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period']].dtypes

KeyField
Site_SAP             object
Summary_Portfolio    object
Material             object
Year_Period          object
dtype: object

In [153]:
excess_inventory[['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period']].dtypes

Site_SAP             object
Summary_Portfolio    object
Material             object
Year_Period          object
dtype: object

In [162]:
y = pd.merge(demand_plan, excess_inventory, 
             on=['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period'], 
             how='left')
demand_plan['Excess_Inventory'] = demand_plan['Excess_Inventory'].fillna(0)
y

,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,Material,Uom,Available,Total,Excess_Inventory
0,North,5502,2020_12,Aseptic,203380,ZUC,1003.0,1003.0,0.0
1,North,5502,2020_12,Aseptic,203381,ZUC,1188.0,1188.0,0.0
2,North,5502,2020_12,Aseptic,203475,ZUC,1192.0,1192.0,0.0
3,North,5502,2020_12,Aseptic,211313,ZUC,4112.0,4112.0,0.0
4,North,5502,2020_12,Aseptic,214633,ZUC,1565.0,1565.0,0.0
...,...,...,...,...,...,...,...,...,...
70346,South,5559,2022_04,Water,448586,ZUC,5589.0,5589.0,0.0
70347,South,5559,2022_04,Water,451493,ZUC,1917.0,1917.0,0.0
70348,South,5559,2022_04,Water,466484,ZUC,4687.0,4687.0,0.0
70349,South,5559,2022_04,Water,466485,ZUC,5582.0,5582.0,0.0


In [161]:
y

,Level2Region,Site_SAP,Year_Period,Summary_Portfolio,Material,Uom,Available,Total,Excess_Inventory
0,North,5502,2020_12,Aseptic,203380,ZUC,1003.0,1003.0,NaN
1,North,5502,2020_12,Aseptic,203381,ZUC,1188.0,1188.0,NaN
2,North,5502,2020_12,Aseptic,203475,ZUC,1192.0,1192.0,NaN
3,North,5502,2020_12,Aseptic,211313,ZUC,4112.0,4112.0,NaN
4,North,5502,2020_12,Aseptic,214633,ZUC,1565.0,1565.0,NaN
...,...,...,...,...,...,...,...,...,...
70346,South,5559,2022_04,Water,448586,ZUC,5589.0,5589.0,0.0
70347,South,5559,2022_04,Water,451493,ZUC,1917.0,1917.0,0.0
70348,South,5559,2022_04,Water,466484,ZUC,4687.0,4687.0,0.0
70349,South,5559,2022_04,Water,466485,ZUC,5582.0,5582.0,0.0


In [186]:
excess_inventory.loc[(excess_inventory.Material == '215671') 
                     & (excess_inventory.Year_Period == '2021_03')
                    ]

,Site_SAP,Summary_Portfolio,Material,Year_Period,Excess_Inventory
4600,5501,LPET,215671,2021_03,0.0
13996,5502,LPET,215671,2021_03,181413.0
29811,5506,LPET,215671,2021_03,0.0
38579,5556,LPET,215671,2021_03,0.0
47466,5558,LPET,215671,2021_03,0.0
56002,5559,LPET,215671,2021_03,0.0


In [179]:
iP_Summarised = pd.read_pickle(os.path.join(output_pickle_dir, 'iP_Summarised'))

iP_Summarised = iP_Summarised[(iP_Summarised.Material == '215671') 
                     & (iP_Summarised.Year_Period == '2021_02')
             & (iP_Summarised.KeyField == 'ZUC_StockVsTarget')]

iP_Summarised

,Table,Level2Region,Site_SAP,SNPPL01,KeyField,Value,Uom,SiteCountry,Summary_Portfolio,Material,Line_Portfolio,Year_Period
84898,Inventory_MaterialLevel,North,5502,LARGE PET >=1L (IH),ZUC_StockVsTarget,181413.0,ZUC,GB,LPET,215671,LPET,2021_02
230706,Inventory_MaterialLevel,North,5558,LARGE PET >=1L (IH),ZUC_StockVsTarget,0.0,ZUC,GB,LPET,215671,LPET,2021_02
628864,Inventory_MaterialLevel,South,5501,LARGE PET >=1L (IH),ZUC_StockVsTarget,0.0,ZUC,GB,LPET,215671,LPET,2021_02
771121,Inventory_MaterialLevel,South,5506,LARGE PET >=1L (IH),ZUC_StockVsTarget,0.0,ZUC,GB,LPET,215671,LPET,2021_02
904090,Inventory_MaterialLevel,South,5556,LARGE PET >=1L (IH),ZUC_StockVsTarget,0.0,ZUC,GB,LPET,215671,LPET,2021_02
1043550,Inventory_MaterialLevel,South,5559,LARGE PET >=1L (IH),ZUC_StockVsTarget,0.0,ZUC,GB,LPET,215671,LPET,2021_02


In [180]:
iP_Summarised['Value'] = iP_Summarised['Value'].astype(float)

In [181]:
pivot_cols = ['KeyField']
pivot_rows = ['Site_SAP', 'Summary_Portfolio', 'Material', 'Year_Period']
value_col = ['Value']
agg_func = np.sum
filter_by = {'Uom': 'ZUC'}

df = iP_Summarised[iP_Summarised.KeyField.isin(['ZUC_StockVsTarget'])]

inventory_excess = pd.pivot_table(df, values=value_col, index=pivot_rows, columns=pivot_cols, aggfunc=agg_func)

inventory_excess = pd.DataFrame(inventory_excess.to_records())

inventory_excess

,Site_SAP,Summary_Portfolio,Material,Year_Period,"('Value', 'ZUC_StockVsTarget')"
0,5501,LPET,215671,2021_02,0.0
1,5502,LPET,215671,2021_02,181413.0
2,5506,LPET,215671,2021_02,0.0
3,5556,LPET,215671,2021_02,0.0
4,5558,LPET,215671,2021_02,0.0
5,5559,LPET,215671,2021_02,0.0


In [182]:
inventory_excess.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in inventory_excess.columns]

num = inventory_excess._get_numeric_data()
num[num < 0] = 0

# shift period by 1 as excess inventory in this period will be supply in next period
inventory_excess.Year_Period = [ get_next_period(x) for x in  inventory_excess.Year_Period ]

#inventory_excess.columns = [pivot_rows + ['Excess_Inventory']]

inventory_excess.rename(columns={'ZUC_StockVsTarget':'Excess_Inventory'}, inplace=True)

inventory_excess

,Site_SAP,Summary_Portfolio,Material,Year_Period,Excess_Inventory
0,5501,LPET,215671,2021_03,0.0
1,5502,LPET,215671,2021_03,181413.0
2,5506,LPET,215671,2021_03,0.0
3,5556,LPET,215671,2021_03,0.0
4,5558,LPET,215671,2021_03,0.0
5,5559,LPET,215671,2021_03,0.0


In [ ]:



iP_Summarised = pd.read_pickle(os.path.join(INPUT_PICKLE_DIR, 'iP_Summarised')).round(2)
iP_Summarised['Value'] = iP_Summarised['Value'].astype(float)

df = iP_Summarised[iP_Summarised.KeyField.isin(['ZUC_StockVsTarget'])]

inventory_excess = pd.pivot_table(df, values=value_col, index=pivot_rows, columns=pivot_cols, aggfunc=agg_func)

inventory_excess = pd.DataFrame(inventory_excess.to_records())

inventory_excess.columns = [hdr.replace("('Value', '", "").replace("')", "") for hdr in inventory_excess.columns]

num = inventory_excess._get_numeric_data()
num[num < 0] = 0

# shift period by 1 as excess inventory in this period will be supply in next period
inventory_excess.Year_Period = [ get_next_period(x) for x in  inventory_excess.Year_Period ]

#inventory_excess.columns = [pivot_rows + ['Excess_Inventory']]

inventory_excess.rename(columns={'ZUC_StockVsTarget':'Excess_Inventory'}, inplace=True)

return inventory_excess